In [ ]:
import pandas as pd

file_path = "../CheXpertPlus/df_chexpert_plus_240401.csv"

df = pd.read_csv(file_path)


In [2]:
print(len(df))
print(df.columns)
print(df["split"].unique())

223462
Index(['path_to_image', 'path_to_dcm', 'frontal_lateral', 'ap_pa',
       'deid_patient_id', 'patient_report_date_order', 'report',
       'section_narrative', 'section_clinical_history', 'section_history',
       'section_comparison', 'section_technique', 'section_procedure_comments',
       'section_findings', 'section_impression', 'section_end_of_impression',
       'section_summary', 'section_accession_number', 'age', 'sex', 'race',
       'ethnicity', 'interpreter_needed', 'insurance_type', 'recent_bmi',
       'deceased', 'split'],
      dtype='object')
['train' 'valid']


In [3]:
import math
import re

Report_Map = {}

for i in range(len(df)):
    path = df["path_to_image"][i]

    report = df["section_impression"][i]

    # Some reports are Nan
    if isinstance(report, float):
        if math.isnan(report):
            report = None
        else:
            print("error!")
    else:
        report = re.sub(r'\s+', ' ', report).strip()

    Report_Map[path] = report


print(len(Report_Map))


223462


In [4]:
import json

file_path = "TestSet_VQA_ReportGeneration"

with open(f"{file_path}.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

print(len(data))


1000


In [5]:

for i in range(len(data)):
    historical_image_path = data[i]["historical_image_path"].replace("png", "jpg")
    image_path = data[i]["image_path"].replace("png", "jpg")

    historical_report = Report_Map[historical_image_path]
    report = Report_Map[image_path]

    # print(historical_report)
    # print(report)


    reference_images = data[i]["reference_images"]
    reference_reports = []
    for k in range(len(reference_images)):
        img = reference_images[k].replace("png", "jpg")
        rep = Report_Map[img]
        reference_reports.append(rep)
    
    # print(reference_reports)


    hist_reference_images = data[i]["hist_reference_images"]
    hist_reference_reports = []
    for k in range(len(hist_reference_images)):
        img = hist_reference_images[k].replace("png", "jpg")
        rep = Report_Map[img]
        hist_reference_reports.append(rep)

    # print(hist_reference_reports)


    data[i]["historical_report"] = historical_report
    data[i]["report"] = report
    data[i]["reference_reports"] = reference_reports
    data[i]["hist_reference_reports"] = hist_reference_reports


print(len(data))

1000


In [6]:
import re

def normalize_report(text):
    text = text.lower()

    sentences = re.split(r'([.!?])', text)

    def smart_capitalize(s):
        for i, c in enumerate(s):
            if c.isalpha():
                return s[:i] + c.upper() + s[i+1:]
        return s

    normalized = [smart_capitalize(s) for s in sentences]

    return ''.join(normalized)


In [7]:

for i in range(len(data)):
    
    report = data[i]["report"]
    data[i]["report"] = normalize_report(report)
    # print(data[i]["report"])

    historical_report = data[i]["historical_report"]
    data[i]["historical_report"] = normalize_report(historical_report)
    # print(data[i]["historical_report"])



    reference_reports = data[i]["reference_reports"]

    for k in range(len(reference_reports)):
        reference_reports[k] = normalize_report(reference_reports[k])

    data[i]["reference_reports"] = reference_reports
    # print(reference_reports)



    hist_reference_reports = data[i]["hist_reference_reports"]

    for k in range(len(hist_reference_reports)):
        hist_reference_reports[k] = normalize_report(hist_reference_reports[k])
    
    data[i]["hist_reference_reports"] = hist_reference_reports
    # print(hist_reference_reports)



In [8]:
save_file = f"{file_path}_Final.json"

with open(save_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
